In [1]:
import pandas as pd
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import SingleIntervalTicker, LinearAxis, ColumnDataSource, HoverTool
from bokeh.palettes import Spectral4, Dark2
from bokeh.models import LinearAxis, Range1d, Legend
from bokeh.layouts import column
import psycopg2

import os
import json
from talib import abstract

In [2]:
def get_stock_price(date, stock_ids):
    conn_params = {
    "host" : "localhost",
    "database" : "Fin_proj",
    "user" : "postgres",
    "password" : "nckumark"
    }
    sql = """   SELECT *
                FROM daily_stock_price
                WHERE date >= (%s) AND \
                stock_id = ANY(%s);
                """
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**conn_params)
        # create a new cursor
        cur = conn.cursor()
        # execute the SQL statement
        cur.execute(sql, (date, stock_ids))
        rows = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return pd.DataFrame(rows, columns=column_names)

In [3]:
date = '2023/01/01'
stock_ids = '2330'
Stock_df = get_stock_price(date, [stock_ids])

In [5]:
Stock_df.head()

,date,stock_id,volume,value,open,high,low,close,spread,turnover
0,2023-01-04,2330,20626874,9310050329,449.50,455.00,448.50,449.50,-3.50,18233
1,2023-01-03,2330,15311364,6871973708,446.00,453.50,443.00,453.00,4.50,22581
2,2023-01-05,2330,23972099,10972616269,459.00,459.50,455.00,458.50,9.00,20752
3,2023-01-10,2330,34785370,16867391241,486.00,487.00,483.00,486.00,5.00,36975
4,2023-01-06,2330,21313593,9745142549,455.00,459.50,455.00,458.50,0.00,16635


In [4]:
def calculate_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    df = df.sort_index(inplace=False)
    df['MA5_price']=df['close'].rolling(5).mean()
    df['MA20_price']=df['close'].rolling(20).mean()
    df['MA60_price']=df['close'].rolling(60).mean()

    df['MA5_vol']=df['value'].rolling(5).mean()
    df['MA20_vol']=df['value'].rolling(20).mean()
    df['MA60_vol']=df['value'].rolling(60).mean()
    return df

In [5]:
def calculate_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    df = df.sort_index(inplace=False)
    df['MA5']=df['close'].rolling(5).mean()
    df['MA10']=df['close'].rolling(10).mean()
    df['MA20']=df['close'].rolling(20).mean()
    df['MA60']=df['close'].rolling(60).mean()
    df['MA120']=df['close'].rolling(120).mean()
    df['volume']=df['volume']/1000
    
    RSI = pd.DataFrame(abstract.RSI(df, timeperiod=12),columns=['RSI_12'])
    RSI['RSI_36']=abstract.RSI(df, timeperiod=36)
    RSI=RSI.to_dict()
    STOCH = abstract.STOCH(df).to_dict()
    MACD=abstract.MACD(df).to_dict()
    basic=df.iloc[-1,:2].to_dict()
    df=df.drop(columns=['stock_id']).to_dict()
    data={'basic':basic,'price_df':df,'RSI':RSI,'STOCH' :STOCH,'MACD':MACD }
    
    return data 

In [6]:
def technical_chart(json_df):
    # get data
    stock_data = json_df
    df = pd.DataFrame(stock_data['price_df']).reset_index().rename(columns={'index': 'date'})
    df = df.reset_index()
    inc = df.close > df.open
    dec = df.open > df.close
    inc_data = df[inc]
    dec_data = df[dec]
    df_source = ColumnDataSource(df)
    inc_source = ColumnDataSource(inc_data)
    dec_source = ColumnDataSource(dec_data)

    # set hover
    hover = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("close", "@open"),
            ("open", "@close"),
            ("high", "@high"),
            ("low", "@low"),
            ("volume", "@volume")
        ],
        formatters={"@date": "datetime"}
    )

    hover_rsi_kd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("RSI_12", "@RSI_12"),
            ("RSI_36", "@RSI_36"),
            ("slowk", "@slowk"),
            ("slowd", "@slowd"),

        ],
        formatters={"@date": "datetime"}

    )

    hover_macd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("macd", "@macd"),
            ("macdsignal", "@macdsignal"),
            ("macdhist", "@macdhist"),
        ],
        formatters={"@date": "datetime"}

    )

    # set figure data
    basic_data = stock_data['basic']
    title = basic_data['stock_id']  + ' ' + 'technical_chart'
    x_end = len(df)
    show_init_num = 120
    x_start = x_end - show_init_num
    interval_freq = show_init_num / 12
    y_start = float(df['close'].min()) * 0.95
    y_end = float(df['close'].max()) * 1.05
    p1 = figure(min_width=1000, title=title, min_height=500, x_range=(x_start, x_end), y_range=(y_start, y_end),
                tools=[hover, "pan,zoom_in,zoom_out,crosshair,reset,save"], toolbar_location="above" ,y_axis_label="price")
    p2 = figure(min_width=1000, title='RSI&KD', min_height=250, x_range=(x_start, x_end),
                background_fill_color="#fafafa", tools=[hover_rsi_kd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
                toolbar_location="above")
    p3 = figure(min_width=1000, title='MACD', min_height=250, x_range=(x_start, x_end),
                background_fill_color="#fafafa", tools=[hover_macd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
                toolbar_location="above")

    for fig in [p1, p2, p3]:
        fig.title.text_font_size = '16pt'

        # map dataframe indices to date strings and use as label overrides
        fig.xaxis.major_label_overrides = {
            i: date.strftime('%Y/%m/%d') for i, date in enumerate(pd.to_datetime(df["date"]))
            # pd.date_range(start='3/1/2000', end='1/08/2018')
        }
        fig.xaxis.ticker = SingleIntervalTicker(interval=interval_freq)

    # set k bar chart
    # use the *indices* for x-axis coordinates, overrides will print better labels
    p1.segment('index', 'high', 'index', 'low', color="black", source=df_source)
    p1.vbar('index', 0.5, 'open', 'close', fill_color="#eb2409", line_color="black", source=inc_source)
    p1.vbar('index', 0.5, 'open', 'close', fill_color="#00995c", line_color="black", source=dec_source)

    # set ma line
    ma_legend_items = []
    for ma_name, color in zip(["MA5", "MA10", "MA20", "MA60", "MA120"], Dark2[5]):
        ma_df = df[['index', 'date', 'close', 'open', 'high', 'low', 'volume', ma_name]]
        source = ColumnDataSource(ma_df)
        ma_line = p1.line(x="index", y=ma_name, line_width=2, color=color, alpha=0.8,
                          muted_color=color, muted_alpha=0.2, source=source)
        ma_legend_items.append((ma_name, [ma_line]))

    # set ma legend
    legend = Legend(items=ma_legend_items, location=(0, 250))
    p1.add_layout(legend, 'left')

    # set twinx for volume
    y2_start = float(df['volume'].min()) * 0.95
    y2_end = float(df['volume'].max()) * 2
    p1.extra_y_ranges = {"vol": Range1d(y2_start, y2_end)}
    p1.vbar('index', 0.5, 'volume', y_range_name='vol', color='blue', alpha=0.2, source=df_source)
    p1.add_layout(LinearAxis(y_range_name="vol" ,axis_label="vol"), 'right')

    # RSI Chart
    rsi_df = pd.DataFrame(stock_data['RSI']).reset_index().rename(columns={'index': 'date'})
    kd_df = pd.DataFrame(stock_data['STOCH']).reset_index().rename(columns={'index': 'date'})
    rsi_kd_df = pd.concat([rsi_df, kd_df], axis=1)
    rsi_source = ColumnDataSource(rsi_kd_df)

    rsi_kd_legend_items = []
    for index_name, color in zip(["RSI_12", "RSI_36", "slowk", "slowd"], Spectral4):
        index_line = p2.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
                             muted_alpha=0.2, source=rsi_source)

        rsi_kd_legend_items.append((index_name, [index_line]))

    # set rsi_kd legend
    legend = Legend(items=rsi_kd_legend_items, location=(0, 50))
    p2.add_layout(legend, 'left')

    # MACD Chart
    macd = pd.DataFrame(stock_data['MACD']).reset_index().rename(columns={'index': 'date'})
    macd = macd.reset_index()
    macd_source = ColumnDataSource(macd)

    macd_legend_items = []
    for index_name, color in zip(["macd", "macdsignal", "macdhist"], Dark2[3]):
        if index_name == "macdhist":
            index_line = p3.vbar('index', 0.5, index_name, color=color, alpha=0.8, muted_color=color, muted_alpha=0.2,
                                 source=macd_source)
        else:
            index_line = p3.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
                                 muted_alpha=0.2, source=macd_source)

        macd_legend_items.append((index_name, [index_line]))

    # set macd legend
    legend = Legend(items=macd_legend_items, location=(0, 50))
    p3.add_layout(legend, 'left')

    # set legend mode
    for fig in [p1, p2, p3]:
        # set legend
        fig.legend.label_text_font_size = '8pt'
        # use hide or mute
        fig.legend.click_policy = "hide"
        #     fig.add_layout(legend, 'left')
    
    # use brower output
#     output_file("candlestick.html", title="candlestick.py example")
    # use jupyter output
    #output_notebook()
    # use columns to control all fig  locations,you could try row method.
    show(p1)
    #show(column(p1, p2, p3))

In [7]:
def technical_chart(json_df):
    # get data
    stock_data = json_df
    df = pd.DataFrame(json_df['price_df']).reset_index().rename(columns={'index': 'date'})
    inc = df.close > df.open
    dec = df.open > df.close
    inc_data = df[inc]
    dec_data = df[dec]
    df_source = ColumnDataSource(df)
    inc_source = ColumnDataSource(inc_data)
    dec_source = ColumnDataSource(dec_data)

    # set hover
    hover = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("close", "@open"),
            ("open", "@close"),
            ("high", "@high"),
            ("low", "@low"),
            ("volume", "@volume")
        ],
        formatters={"@date": "datetime"}
    )

    hover_rsi_kd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("RSI_12", "@RSI_12"),
            ("RSI_36", "@RSI_36"),
            ("slowk", "@slowk"),
            ("slowd", "@slowd"),

        ],
        formatters={"@date": "datetime"}

    )

    hover_macd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("macd", "@macd"),
            ("macdsignal", "@macdsignal"),
            ("macdhist", "@macdhist"),
        ],
        formatters={"@date": "datetime"}

    )

    # set figure data
    basic_data = stock_data['basic']
    title = basic_data['stock_id']  + ' ' + 'technical_chart'
    x_end = len(df)
    show_init_num = 120
    x_start = x_end - show_init_num
    interval_freq = show_init_num / 12
    y_start = float(df['close'].min()) * 0.95
    y_end = float(df['close'].max()) * 1.05
    p1 = figure(min_width=1000, title=title, min_height=500,
                tools=[hover, "pan,zoom_in,zoom_out,crosshair,reset,save"], toolbar_location="above" ,y_axis_label="price")
    # p2 = figure(min_width=1000, title='RSI&KD', min_height=250, x_range=(x_start, x_end),
    #             background_fill_color="#fafafa", tools=[hover_rsi_kd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
    #             toolbar_location="above")
    # p3 = figure(min_width=1000, title='MACD', min_height=250, x_range=(x_start, x_end),
    #             background_fill_color="#fafafa", tools=[hover_macd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
    #             toolbar_location="above")

    # for fig in [p1, p2, p3]:
    #     fig.title.text_font_size = '16pt'

    #     # map dataframe indices to date strings and use as label overrides
    #     fig.xaxis.major_label_overrides = {
    #         i: date.strftime('%Y/%m/%d') for i, date in enumerate(pd.to_datetime(df["date"]))
    #         # pd.date_range(start='3/1/2000', end='1/08/2018')
    #     }
    #     fig.xaxis.ticker = SingleIntervalTicker(interval=interval_freq)

    # set k bar chart
    # use the *indices* for x-axis coordinates, overrides will print better labels
    w = 24*60*60*1000 # half day in ms
    p1.segment('date', 'high', 'date', 'low', color="black", source=df_source)
    p1.vbar('date', w, 'open', 'close', fill_color="#eb2409", line_color="black", source=inc_source)
    p1.vbar('date', w, 'open', 'close', fill_color="#00995c", line_color="black", source=dec_source)

    # # set ma line
    # ma_legend_items = []
    # for ma_name, color in zip(["MA5", "MA10", "MA20", "MA60", "MA120"], Dark2[5]):
    #     ma_df = df[['index', 'date', 'close', 'open', 'high', 'low', 'volume', ma_name]]
    #     source = ColumnDataSource(ma_df)
    #     ma_line = p1.line(x="index", y=ma_name, line_width=2, color=color, alpha=0.8,
    #                       muted_color=color, muted_alpha=0.2, source=source)
    #     ma_legend_items.append((ma_name, [ma_line]))

    # # set ma legend
    # legend = Legend(items=ma_legend_items, location=(0, 250))
    # p1.add_layout(legend, 'left')

    # # set twinx for volume
    # y2_start = float(df['volume'].min()) * 0.95
    # y2_end = float(df['volume'].max()) * 2
    # p1.extra_y_ranges = {"vol": Range1d(y2_start, y2_end)}
    # p1.vbar('index', 0.5, 'volume', y_range_name='vol', color='blue', alpha=0.2, source=df_source)
    # p1.add_layout(LinearAxis(y_range_name="vol" ,axis_label="vol"), 'right')

    # # RSI Chart
    # rsi_df = pd.DataFrame(stock_data['RSI']).reset_index().rename(columns={'index': 'date'})
    # kd_df = pd.DataFrame(stock_data['STOCH']).reset_index().rename(columns={'index': 'date'})
    # rsi_kd_df = pd.concat([rsi_df, kd_df], axis=1)
    # rsi_source = ColumnDataSource(rsi_kd_df)

    # rsi_kd_legend_items = []
    # for index_name, color in zip(["RSI_12", "RSI_36", "slowk", "slowd"], Spectral4):
    #     index_line = p2.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
    #                          muted_alpha=0.2, source=rsi_source)

    #     rsi_kd_legend_items.append((index_name, [index_line]))

    # # set rsi_kd legend
    # legend = Legend(items=rsi_kd_legend_items, location=(0, 50))
    # p2.add_layout(legend, 'left')

    # # MACD Chart
    # macd = pd.DataFrame(stock_data['MACD']).reset_index().rename(columns={'index': 'date'})
    # macd = macd.reset_index()
    # macd_source = ColumnDataSource(macd)

    # macd_legend_items = []
    # for index_name, color in zip(["macd", "macdsignal", "macdhist"], Dark2[3]):
    #     if index_name == "macdhist":
    #         index_line = p3.vbar('index', 0.5, index_name, color=color, alpha=0.8, muted_color=color, muted_alpha=0.2,
    #                              source=macd_source)
    #     else:
    #         index_line = p3.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
    #                              muted_alpha=0.2, source=macd_source)

    #     macd_legend_items.append((index_name, [index_line]))

    # # set macd legend
    # legend = Legend(items=macd_legend_items, location=(0, 50))
    # p3.add_layout(legend, 'left')

    # # set legend mode
    # for fig in [p1, p2, p3]:
    #     # set legend
    #     fig.legend.label_text_font_size = '8pt'
    #     # use hide or mute
    #     fig.legend.click_policy = "hide"
    #     #     fig.add_layout(legend, 'left')
    
    # use brower output
#     output_file("candlestick.html", title="candlestick.py example")
    # use jupyter output
    #output_notebook()
    # use columns to control all fig  locations,you could try row method.
    show(p1)
    #show(column(p1, p2, p3))

In [8]:
json_df = calculate_data(Stock_df)
a = technical_chart(json_df)

SerializationError: can't serialize <class 'decimal.Decimal'>

In [9]:
import pandas as pd

from bokeh.plotting import figure, show
from bokeh.sampledata.stocks import MSFT

df = pd.DataFrame(MSFT)[60:120]
df["date"] = pd.to_datetime(df["date"])

inc = df.close > df.open
dec = df.open > df.close
w = 16*60*60*1000 # milliseconds

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, width=2000, height=400,
           title="MSFT Candlestick", background_fill_color="#efefef")
p.xaxis.major_label_orientation = 0.8 # radians

p.segment(df.date, df.high, df.date, df.low, color="black")

p.vbar(df.date[dec], w, df.open[dec], df.close[dec], color="#eb3c40")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="white",
       line_color="#49a3a3", line_width=2)

show(p)